In this notebook, I will generate a combined anndata object from all of the samples\
in the Zheng et al 2022 mouse stroke scRNA-Seq dataset.

In [1]:
import os

In [2]:
os.getcwd()

'/fast/AG_Bunina/Yusuf/Project_Endothelial_and_Stroke/Datasets/GENE_EXPRESSION/MOUSE/STROKE/2022_Zheng_K_et_al/28_10_24'

In [3]:
!ls

PART_1.ipynb
initial_generation_of_anndata_object_with_some_errors_and_solutions.ipynb


In [4]:
import scanpy as sc

In [5]:
sc.settings.set_figure_params(dpi=50, facecolor="white")

In [6]:
os.listdir(os.path.join('..', 'GSE174574_RAW'))

['GSM5319989_sham3_barcodes.tsv.gz',
 'GSM5319990_MCAO1_barcodes.tsv.gz',
 'GSM5319987_sham1_features.tsv.gz',
 'GSM5319988_sham2_matrix.mtx.gz',
 'GSM5319992_MCAO3_barcodes.tsv.gz',
 'GSM5319987_sham1_barcodes.tsv.gz',
 'GSM5319989_sham3_matrix.mtx.gz',
 'GSM5319990_MCAO1_matrix.mtx.gz',
 'GSM5319991_MCAO2_features.tsv.gz',
 'GSM5319990_MCAO1_features.tsv.gz',
 'GSM5319991_MCAO2_barcodes.tsv.gz',
 'GSM5319992_MCAO3_matrix.mtx.gz',
 'GSM5319987_sham1_matrix.mtx.gz',
 'GSM5319988_sham2_barcodes.tsv.gz',
 'GSM5319989_sham3_features.tsv.gz',
 'GSM5319988_sham2_features.tsv.gz',
 'GSM5319991_MCAO2_matrix.mtx.gz',
 'GSM5319992_MCAO3_features.tsv.gz']

In [8]:
os.path.join('..', 'GSE174574_RAW')

'../GSE174574_RAW'

Following test function to import a file for one sample resulted in an error:

In [9]:
# Path to directory and prefix to files
path = os.path.join('..', 'GSE174574_RAW')
prefix = 'GSM5319988_sham2_'

# Attempt reading in legacy format by setting `var_names` and checking if `feature_types` exists
try:
    adata_test = sc.read_10x_mtx(path=path, var_names='gene_symbols', prefix=prefix)
except KeyError:
    # Fallback option if the file format is legacy and lacks feature_types
    print("KeyError occurred, trying without feature_types.")
    adata_test = sc.read_10x_mtx(path=path, var_names='gene_symbols', prefix=prefix, gex_only=False)

KeyError occurred, trying without feature_types.


KeyError: 2

To address this error, previously, I imported the features file for the sham_1 sample and \
adjusted its column structure to match the expectations of the anndata import function.\
Below, I will use the same codes to adjust for sham_2 sample and then write a loop \
to adjust all of the remaining samples.

Check the reading function first:

In [13]:
import pandas as pd
import os

# Load features.tsv file and check columns
features_file = os.path.join('..', 'GSE174574_RAW', 'GSM5319988_sham2_features.tsv.gz')
features = pd.read_csv(features_file, sep='\t', header=None)

In [14]:
features

,0,1
0,ENSMUSG00000051951,Xkr4
1,ENSMUSG00000089699,Gm1992
2,ENSMUSG00000102343,Gm37381
3,ENSMUSG00000025900,Rp1
4,ENSMUSG00000109048,Rp1
...,...,...
27993,ENSMUSG00000079808,AC168977.1
27994,ENSMUSG00000095041,PISD
27995,ENSMUSG00000063897,DHRSX
27996,ENSMUSG00000096730,Vmn2r122


In [15]:
# It works. 
# Now, I will read and adjust the features dataframe:

In [16]:
import pandas as pd
import os

# Load features.tsv file and check columns
features_file = os.path.join('..', 'GSE174574_RAW', 'GSM5319988_sham2_features.tsv.gz')
features = pd.read_csv(features_file, sep='\t', header=None)

# If there are only two columns, add a placeholder third column
if features.shape[1] == 2:
    features[2] = 'Gene Expression'
    features.to_csv(features_file, sep='\t', index=False, header=False)

# Retry loading the matrix
import scanpy as sc
adata_sham2 = sc.read_10x_mtx(path=os.path.join('..', 'GSE174574_RAW'), var_names='gene_symbols', prefix='GSM5319988_sham2_')


In [17]:
adata_sham2

AnnData object with n_obs × n_vars = 8540 × 27998
    var: 'gene_ids', 'feature_types'

In [18]:
adata_sham2.obs

""
AAACCTGAGCGAAGGG-1
AAACCTGAGCTAACAA-1
AAACCTGAGGCTAGCA-1
AAACCTGAGGTGCACA-1
AAACCTGAGTTTAGGA-1
...
TTTGTCATCCCTTGCA-1
TTTGTCATCGAGCCCA-1
TTTGTCATCGTGGACC-1
TTTGTCATCGTTGCCT-1


In [19]:
adata_sham2.var

,gene_ids,feature_types
Xkr4,ENSMUSG00000051951,Gene Expression
Gm1992,ENSMUSG00000089699,Gene Expression
Gm37381,ENSMUSG00000102343,Gene Expression
Rp1,ENSMUSG00000025900,Gene Expression
Rp1-1,ENSMUSG00000109048,Gene Expression
...,...,...
AC168977.1,ENSMUSG00000079808,Gene Expression
PISD,ENSMUSG00000095041,Gene Expression
DHRSX,ENSMUSG00000063897,Gene Expression
Vmn2r122,ENSMUSG00000096730,Gene Expression


In [23]:
# read also the sham_1 data:

# Retry loading the matrix
import scanpy as sc
adata_sham1 = sc.read_10x_mtx(path=os.path.join('..', 'GSE174574_RAW'), var_names='gene_symbols', prefix='GSM5319987_sham1_')


In [24]:
adata_sham1

AnnData object with n_obs × n_vars = 8771 × 27998
    var: 'gene_ids', 'feature_types'

In [28]:
adata_sham1.obs

""
AAACCTGAGCACAGGT-1
AAACCTGAGCGTCAAG-1
AAACCTGAGGGTATCG-1
AAACCTGAGTTGAGAT-1
AAACCTGCACCTCGGA-1
...
TTTGTCAGTTATCACG-1
TTTGTCAGTTCTCATT-1
TTTGTCATCAGTGTTG-1
TTTGTCATCCTAGAAC-1


In [29]:
adata_sham1.var

,gene_ids,feature_types
Xkr4,ENSMUSG00000051951,Gene Expression
Gm1992,ENSMUSG00000089699,Gene Expression
Gm37381,ENSMUSG00000102343,Gene Expression
Rp1,ENSMUSG00000025900,Gene Expression
Rp1-1,ENSMUSG00000109048,Gene Expression
...,...,...
AC168977.1,ENSMUSG00000079808,Gene Expression
PISD,ENSMUSG00000095041,Gene Expression
DHRSX,ENSMUSG00000063897,Gene Expression
Vmn2r122,ENSMUSG00000096730,Gene Expression


In [31]:
# Now, I assured that sham_1 and sham_2 data are seamlessly read. Now I will loop over all other sample files.
# Fisrtly, I will adjust their features tsv files. Then, I will generate a merged adata from all samples.

In [32]:
os.listdir(os.path.join('..', 'GSE174574_RAW'))

['GSM5319989_sham3_barcodes.tsv.gz',
 'GSM5319990_MCAO1_barcodes.tsv.gz',
 'GSM5319987_sham1_features.tsv.gz',
 'GSM5319988_sham2_matrix.mtx.gz',
 'GSM5319992_MCAO3_barcodes.tsv.gz',
 'GSM5319987_sham1_barcodes.tsv.gz',
 'GSM5319989_sham3_matrix.mtx.gz',
 'GSM5319990_MCAO1_matrix.mtx.gz',
 'GSM5319991_MCAO2_features.tsv.gz',
 'GSM5319990_MCAO1_features.tsv.gz',
 'GSM5319991_MCAO2_barcodes.tsv.gz',
 'GSM5319992_MCAO3_matrix.mtx.gz',
 'GSM5319987_sham1_matrix.mtx.gz',
 'GSM5319988_sham2_barcodes.tsv.gz',
 'GSM5319989_sham3_features.tsv.gz',
 'GSM5319988_sham2_features.tsv.gz',
 'GSM5319991_MCAO2_matrix.mtx.gz',
 'GSM5319992_MCAO3_features.tsv.gz']

In [33]:
import pandas as pd
import os

# Directory containing the files
directory = os.path.join('..', 'GSE174574_RAW')

# List of files in the directory
files = os.listdir(directory)   #The script accesses the GSE174574_RAW directory and lists all files in it.

# Loop through the files and process them
for file in files:
    # Exclude sham1 and sham2 and only process features.tsv.gz files
    if 'features.tsv.gz' in file and 'sham1' not in file and 'sham2' not in file: 
        features_file = os.path.join(directory, file)
        # Load the features.tsv file
        features = pd.read_csv(features_file, sep='\t', header=None)

        # If there are only two columns, add a placeholder third column
        if features.shape[1] == 2:                      # If the file has only two columns, a third column labeled 'Gene Expression' is added.
            features[2] = 'Gene Expression'
            features.to_csv(features_file, sep='\t', index=False, header=False)

        print(f"Processed {file}")

Processed GSM5319991_MCAO2_features.tsv.gz
Processed GSM5319990_MCAO1_features.tsv.gz
Processed GSM5319989_sham3_features.tsv.gz
Processed GSM5319992_MCAO3_features.tsv.gz


In [36]:
# View the total files to check all files are adjusted.

! ls ../GSE174574_RAW/*_features.tsv.gz

../GSE174574_RAW/GSM5319987_sham1_features.tsv.gz
../GSE174574_RAW/GSM5319988_sham2_features.tsv.gz
../GSE174574_RAW/GSM5319989_sham3_features.tsv.gz
../GSE174574_RAW/GSM5319990_MCAO1_features.tsv.gz
../GSE174574_RAW/GSM5319991_MCAO2_features.tsv.gz
../GSE174574_RAW/GSM5319992_MCAO3_features.tsv.gz


In [37]:
# generate an anndata object:

In [38]:
import os
import scanpy as sc

# Directory containing the data
data_dir = os.path.join('..', 'GSE174574_RAW')

# List of files in the directory
files = os.listdir(data_dir)

# Filter for matrix.mtx.gz files
matrix_files = [f for f in files if 'matrix.mtx.gz' in f]

In [39]:
matrix_files

['GSM5319988_sham2_matrix.mtx.gz',
 'GSM5319989_sham3_matrix.mtx.gz',
 'GSM5319990_MCAO1_matrix.mtx.gz',
 'GSM5319992_MCAO3_matrix.mtx.gz',
 'GSM5319987_sham1_matrix.mtx.gz',
 'GSM5319991_MCAO2_matrix.mtx.gz']

In [40]:
# Initialize a list to hold individual AnnData objects
adatas = []

# Process each sample
for matrix_file in matrix_files:
    # Get the sample prefix by stripping '_matrix.mtx.gz'
    sample_prefix = matrix_file.replace('_matrix.mtx.gz', '_')
    
    # Build the path to the directory containing the sample
    sample_dir = os.path.join(data_dir)
    
    # Read the data into an AnnData object
    adata = sc.read_10x_mtx(
        path=sample_dir, 
        var_names='gene_symbols', 
        prefix=sample_prefix
    )
    
    # Add a column in the observation dataframe for sample identity
    adata.obs['sample'] = sample_prefix
    
    # Append to the list
    adatas.append(adata)

In [41]:
adatas

[AnnData object with n_obs × n_vars = 8540 × 27998
     obs: 'sample'
     var: 'gene_ids', 'feature_types',
 AnnData object with n_obs × n_vars = 9980 × 27998
     obs: 'sample'
     var: 'gene_ids', 'feature_types',
 AnnData object with n_obs × n_vars = 11772 × 27998
     obs: 'sample'
     var: 'gene_ids', 'feature_types',
 AnnData object with n_obs × n_vars = 8104 × 27998
     obs: 'sample'
     var: 'gene_ids', 'feature_types',
 AnnData object with n_obs × n_vars = 8771 × 27998
     obs: 'sample'
     var: 'gene_ids', 'feature_types',
 AnnData object with n_obs × n_vars = 11361 × 27998
     obs: 'sample'
     var: 'gene_ids', 'feature_types']

In [44]:
# Concatenate all AnnData objects into a single AnnData object:

# Below code, the "*" operator unpacks a list or iterable into individual elements.

# So, in adatas[0].concatenate(*adatas[1:], ...), it unpacks adatas[1:] so that 
# each AnnData object in the list is passed as a separate argument to the concatenate() method.
# This is important because concatenate function of anndata object takes multiple anndata objects
# to be added to the initial anndata object.

combined_adata = adatas[0].concatenate(*adatas[1:], batch_key='sample', batch_categories=[a.obs['sample'][0] for a in adatas])

# Save the combined AnnData object
combined_adata.write(os.path.join('..', 'GSE174574_combined.h5ad'))

print("Combined AnnData object has been saved!")

/var/tmp/ipykernel_3344333/3797977896.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combined_adata = adatas[0].concatenate(*adatas[1:], batch_key='sample', batch_categories=[a.obs['sample'][0] for a in adatas])
/var/tmp/ipykernel_3344333/3797977896.py:10: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  combined_adata = adatas[0].concatenate(*adatas[1:], batch_key='sample', batch_categories=[a.obs['sample'][0] for a in adatas])


Combined AnnData object has been saved!


In [45]:
combined_adata

AnnData object with n_obs × n_vars = 58528 × 27998
    obs: 'sample'
    var: 'gene_ids', 'feature_types'

In [49]:
combined_adata.obs.value_counts()

sample           
GSM5319990_MCAO1_    11772
GSM5319991_MCAO2_    11361
GSM5319989_sham3_     9980
GSM5319987_sham1_     8771
GSM5319988_sham2_     8540
GSM5319992_MCAO3_     8104
Name: count, dtype: int64

In [50]:
import session_info

In [51]:
session_info.show()